In [2]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

In [3]:
#use to to look at the data for 1 ticker
ticker = yf.Ticker('JPY=X')
df = ticker.history(period = '6mo', interval='1d')
df.head()

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-24 00:00:00+01:00,144.673004,145.947006,144.621002,144.673004,0,0.0,0.0
2023-08-25 00:00:00+01:00,146.067001,146.604996,145.733994,146.067001,0,0.0,0.0
2023-08-28 00:00:00+01:00,146.531006,146.716003,146.278000,146.531006,0,0.0,0.0
2023-08-29 00:00:00+01:00,146.464005,147.343994,145.852005,146.464005,0,0.0,0.0
2023-08-30 00:00:00+01:00,145.988007,146.522995,145.557999,145.988007,0,0.0,0.0


In [4]:
# function to format in a useable df, only keeping the date part of datetime to avoid time-caused errors
# also we found that from yfinanve the 'Date' column was actually an index, so converted it to a column
def get_stock_data_to_df(ticker):
    ticker = yf.Ticker(str(ticker))
    df = ticker.history(period = '6mo', interval='1d')
    df.reset_index(inplace=True)
    df["timestamp_processed"] = [e.strftime('%Y-%m-%d') for e in df["Date"]]
    return df

# making the data stionary to fit the assumption. logging the price for the column given and taking the diffence
# created a new column for that called 'stationary'
def stationaryize(df : pd.DataFrame, time) -> pd.DataFrame:
    df['stationary'] = (np.log(df[time])).diff()
    df = df.dropna(how = 'any', axis = 0, )
    return df


def get_clean_data(ticker, time) -> pd.DataFrame:
    return stationaryize(get_stock_data_to_df(ticker.upper()), time)


# main function here which takes in 2 tickers and the open/close/high/low column, and merges the stationary columnms for each on the timestamp column

def merge_df(ticker1, ticker2, open_or_close):
    df1 = get_clean_data(ticker1, open_or_close)
    # print("df of  " + ticker1 + " " + str(np.shape(df1)))
    # print("df of " + ticker1)
    # print(df1.head())
    df2 = get_clean_data(ticker2, open_or_close)
    # print("df of  " + ticker2 + " " + str(np.shape(df2)))
    # print("df of " + ticker2)
    # print(df2.head())
    # return pd.merge(df1[['stationary']], df2[['stationary']], left_index=True, right_index=True, how='right').rename(columns={'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')
    return pd.merge(df1[['timestamp_processed', 'stationary']], df2[['timestamp_processed', 'stationary']], on='timestamp_processed', how='right').rename(columns = {'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')


In [5]:
merge_df('^TNX', '^GSPC', 'Close')

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,timestamp_processed,^TNX,^GSPC
0,2023-08-25,0.000944,0.006696
1,2023-08-28,-0.006390,0.006245
2,2023-08-29,-0.021599,0.014404
3,2023-08-30,-0.000971,0.003826
4,2023-08-31,-0.006089,-0.001598
...,...,...,...
120,2024-02-16,0.012888,-0.004815
121,2024-02-20,-0.004667,-0.006023
122,2024-02-21,0.011628,0.001263
123,2024-02-22,0.000462,0.020903


In [6]:
# function for the granger test. takes in the tickers, time(open/close/high/low), and lag, and calls merge_df function. returns the test


def granger_causality(ticker1, ticker2, time, lag):
    df = merge_df(ticker1, ticker2, time)
    # if len(df[ticker1]) != len(df[ticker2]):
    # minrows = min(len(df[ticker1]), len(df[ticker2]))
    # df = df.iloc[:,:minrows]
    #print(np.shape(df))
    return grangercausalitytests(df[[ticker1, ticker2]], lag)

In [7]:
granger_causality('TSLA', 'AAPL', 'Open', 4)

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.5088  , p=0.2217  , df_denom=121, df_num=1
ssr based chi2 test:   chi2=1.5462  , p=0.2137  , df=1
likelihood ratio test: chi2=1.5367  , p=0.2151  , df=1
parameter F test:         F=1.5088  , p=0.2217  , df_denom=121, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1019  , p=0.3356  , df_denom=118, df_num=2
ssr based chi2 test:   chi2=2.2972  , p=0.3171  , df=2
likelihood ratio test: chi2=2.2760  , p=0.3205  , df=2
parameter F test:         F=1.1019  , p=0.3356  , df_denom=118, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3542  , p=0.7862  , df_denom=115, df_num=3
ssr based chi2 test:   chi2=1.1272  , p=0.7705  , df=3
likelihood ratio test: chi2=1.1220  , p=0.7718  , df=3
parameter F test:         F=0.3542  , p=0.7862  , df_denom=115, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.5115  , p=0.7274  

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


{1: ({'ssr_ftest': (1.508810846509483, 0.2217044850030784, 121.0, 1),
   'ssr_chi2test': (1.5462193798940154, 0.21369443175380395, 1),
   'lrtest': (1.5366584778173547, 0.21511587012954878, 1),
   'params_ftest': (1.5088108465095091, 0.2217044850030719, 121.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.1018930657770776, 0.3356353928485849, 118.0, 2),
   'ssr_chi2test': (2.2971668998403483, 0.31708561905395877, 2),
   'lrtest': (2.2759791757729317, 0.32046263660060353, 2),
   'params_ftest': (1.1018930657770867, 0.3356353928485826, 118.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (0.354173910386429, 0.7862008605791284, 115.0, 3),
   'ssr_chi2test': (1.1271969669689827, 0.7705130138639765, 3),
   'lrtest': (1.122021554335447, 0.7717608279587573, 3),
   'params_ftest': (0.3541739103864398, 0.7862008605791222, 115.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0.,

In [8]:
granger_causality('^TNX', '^GSPC', 'Close', 4)

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0295  , p=0.8639  , df_denom=121, df_num=1
ssr based chi2 test:   chi2=0.0302  , p=0.8620  , df=1
likelihood ratio test: chi2=0.0302  , p=0.8620  , df=1
parameter F test:         F=0.0295  , p=0.8639  , df_denom=121, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.1353  , p=0.0184  , df_denom=118, df_num=2
ssr based chi2 test:   chi2=8.6212  , p=0.0134  , df=2
likelihood ratio test: chi2=8.3324  , p=0.0155  , df=2
parameter F test:         F=4.1353  , p=0.0184  , df_denom=118, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.2988  , p=0.0230  , df_denom=115, df_num=3
ssr based chi2 test:   chi2=10.4987 , p=0.0148  , df=3
likelihood ratio test: chi2=10.0713 , p=0.0180  , df=3
parameter F test:         F=3.2988  , p=0.0230  , df_denom=115, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.2966  , p=0.0635  

{1: ({'ssr_ftest': (0.029495544568400398, 0.8639267380743556, 121.0, 1),
   'ssr_chi2test': (0.030226839061831812, 0.8619765300826252, 1),
   'lrtest': (0.03022315554017041, 0.8619848559000673, 1),
   'params_ftest': (0.029495544568467397, 0.8639267380742328, 121.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (4.1353497623285405, 0.018373243578831015, 118.0, 2),
   'ssr_chi2test': (8.62115289434594, 0.013425808094056073, 2),
   'lrtest': (8.332436586403901, 0.015510806674234687, 2),
   'params_ftest': (4.135349762328532, 0.018373243578831168, 118.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (3.298770982798562, 0.022969289198747717, 115.0, 3),
   'ssr_chi2test': (10.49869721481977, 0.014769737290802448, 3),
   'lrtest': (10.071315733051392, 0.01796955212821058, 3),
   'params_ftest': (3.2987709827985623, 0.022969289198747717, 115.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
       